In [49]:
import os
from tqdm import tqdm
from keras.utils import load_img, img_to_array, to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
import numpy as np

In [3]:
images_base_path = r'/Users/praneethkumarpalepu/Downloads/animals/images'

In [42]:
def image_dataset_loader(base_path, image_size=(224, 224), test_size=0.2, random_state=0, batch_size= 64):
    images, labels = [], []
    image_file_location, act_label = [], []
    for folder in os.listdir(base_path):
        if not folder.startswith('.'):
            folder_path = os.path.join(base_path, folder)
            for file in os.listdir(folder_path):
                _, ext = os.path.splitext(file)
                if ext.lower() in ['.jpg', '.jpeg', '.png']:
                    file_path = os.path.join(folder_path, file)
                    image_file_location.append(file_path)
                    act_label.append(folder)

    total_files = len(image_file_location)
    quotient = total_files//batch_size
    reminder = total_files % batch_size
    total_batches = quotient+1 if reminder > 0 else quotient

    for batch in tqdm(range(total_batches)):
        start_index = batch * batch_size
        end_index = (batch * batch_size)+batch_size
        if end_index > total_files:
            end_index = end_index - total_files
        image_batch = image_file_location[start_index:end_index]
        label = act_label[start_index:end_index]
        counter= 0
        tmp_image, tmp_labels = [], []
        for file in image_batch:           
            image = load_img(file, target_size=image_size)
            image_arr = img_to_array(image)
            tmp_image.append(image_arr)
            tmp_labels.append(label[counter])
            counter+=1

        images.extend(np.array(tmp_image))
        labels.extend(np.array(tmp_labels))

    images = np.array(images)
    labels = np.array(labels)

    xtr, xte, ytr, yte = train_test_split(images, labels, test_size= test_size, random_state= random_state)

    return (xtr, ytr), (xte, yte)

In [46]:
(xtr, ytr), (xte, yte) = image_dataset_loader(images_base_path, batch_size=128)

100%|███████████████████████████████████████████| 24/24 [00:04<00:00,  5.89it/s]


In [47]:
xtr.shape

(2355, 224, 224, 3)

In [48]:
xte.shape

(589, 224, 224, 3)

In [50]:
encoder = LabelEncoder()

In [51]:
ytr = encoder.fit_transform(ytr)

In [52]:
yte = encoder.transform(yte)

In [53]:
ytr = to_categorical(ytr)

In [54]:
yte = to_categorical(yte)

In [55]:
xtr = xtr.astype('float32')/255
xte = xte.astype('float32')/255

In [59]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

In [60]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
history = model.fit(xtr, ytr, validation_data=(xte, yte), epochs=25, batch_size=256)

Epoch 1/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 58s 5s/step - accuracy: 0.3519 - loss: 1.0910 - val_accuracy: 0.4652 - val_loss: 1.0140
Epoch 2/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 59s 6s/step - accuracy: 0.5453 - loss: 0.9486 - val_accuracy: 0.5637 - val_loss: 0.8187
Epoch 3/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 57s 6s/step - accuracy: 0.5654 - loss: 0.7872 - val_accuracy: 0.4941 - val_loss: 0.9227
Epoch 4/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 59s 6s/step - accuracy: 0.5423 - loss: 0.8370 - val_accuracy: 0.5840 - val_loss: 0.8325
Epoch 5/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 62s 6s/step - accuracy: 0.5741 - loss: 0.7734 - val_accuracy: 0.5365 - val_loss: 0.7909
Epoch 6/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 62s 6s/step - accuracy: 0.5896 - loss: 0.7546 - val_accuracy: 0.5993 - val_loss: 0.7691
Epoch 7/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 63s 6s/step - accuracy: 0.6240 - loss: 0.7086 - val_accuracy: 0.6146 - val_loss: 0.7349
Epoch 8/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 61s 6s/step - accuracy: 0.6470 - loss: 0.6618 - val_accuracy: 0.6486 - val_loss: